Load in data csv and specify output folder

In [1]:
from pathlib import Path

import pylibxc
import numpy as np
import pandas as pd

# path/to/data.csv
data_path = Path('data/comprehensive_search.csv')
# folder for outputs
out_path = Path('out')
out_path.mkdir(exist_ok=True, parents=True)
df = pd.read_csv(data_path)

Generate bibtex file and references for functionals

In [2]:
import re

bib_file = []
ref_names = []
is_combined_xc = []
for xc in df['xc_id']:
  
  if '_xc_' in xc.lower():
    combined_xc = True
  else: 
    combined_xc = False
  is_combined_xc.append(combined_xc)

  func = pylibxc.LibXCFunctional(xc, "polarized")  
  ref_name='\cite{'
  for ref in func.get_bibtex():
    
    latex_cite = ref.splitlines()[0]
    # get latex cite name by removing the following prefixes
    # like @article{, @inproceedings{, @Inbook{, @misc{, etc.
    latex_cite = re.search(r'@.*{(.*)', latex_cite).group(1)  

    if latex_cite == 'Bilc2008_165107,':
        # fix for Bilc2008_165107 ref.
        ref = ref.replace(r'{\'{I}\\textasciitilde {n}iguez}', r'\'I\~niguez') 

    if ref not in bib_file:             
      bib_file.append(ref)
    
    ref_name += latex_cite
    
  ref_name = ref_name[:-1] + '}' 
  ref_names.append(ref_name)

df['Refs.'] = ref_names 
df['combined xc'] = is_combined_xc

# write out bib file
bib_file = "\n".join(bib_file)
out_file = out_path / data_path.name.replace('.csv','.bib')
f = open(out_file, "w")
f.write(bib_file)
f.close()
print('wrote file: ', out_file)


wrote file:  out/comprehensive_search.bib


Create latex tables

In [3]:

# max number of rows in each table (to account for page breaks)
max_table_size=60

table_header = {
    'table_name': '',
    'negativity_check': r'\makecell[c]{$E\C[n]$ \\ non-positivity}',
    'deriv_lower_bd_check': r'\makecell[c]{$E\C[n\g]$ uniform \\ scaling inequality}',
    'deriv_upper_bd_check_1': r'\makecell[c]{$T\C[n]$ \\ upper bound}',
    'second_deriv_check': r'\makecell[c]{concavity of \\ $\lambda^2 E\c[n_{1/\lambda}]$}',
    'lieb_oxford_bd_check_Exc': r'\makecell[c]{LO extension \\ to $E\xc$}',
    'lieb_oxford_bd_check_Uxc': r'\makecell[c]{LO}',
    "deriv_upper_bd_check_2": r'\makecell[c]{conjecture: \\ $T_c \leq -E_c$}',
}

# latex table captions
captions = {
    'GGA': 'GGA functionals: numerical assessment of corresponding local conditions.',
    'MGGA': 'MGGA functionals: numerical assessment of corresponding local conditions.',
    'HYB_GGA': 'Hybrid GGA functionals: numerical assessment of corresponding local conditions.',
    'HYB_MGGA': 'Hybrid MGGA functionals: numerical assessment of corresponding local conditions.',
}

# create a more manageable dataframe to use for the printed table
min_df = df[["xc_id", "xc_name", "type", "Refs.", "condition", "violation fraction"]]
# rearrange the dataframe to have the conditions as columns
table_df = min_df.pivot_table(values='violation fraction', index=min_df['xc_id'], columns='condition', aggfunc='first')
table_df["table_name"] = min_df.groupby('xc_id').apply(lambda x: (x['xc_name'] + "~" + x['Refs.']).iloc[0])
table_df["type"] = min_df.groupby('xc_id').apply(lambda x: x['type'].iloc[0])

sorted_cols = list(table_header)
families = list(captions)
for family in families:

  tex_df = (table_df[table_df['type'] == family]
    .sort_values(by=['table_name']))
  
  # use scientific notation for entries less than 0.001
  tex_df.loc[:, sorted_cols[1:]] = tex_df[sorted_cols[1:]].applymap(lambda x: '{:.0e}'.format(x) if 0 < x < 0.001 else x)

  tex_df = (tex_df.reindex(sorted_cols, axis=1)
    .rename(columns=table_header)
    .replace(0, '0')
    .fillna('---')
  )
  
  # split numpy array where the maximum size of each subarray is 50
  ranges = np.arange(max_table_size, tex_df.shape[0], max_table_size)
  split_tex_dfs = np.array_split(tex_df, ranges)

  for i, split_tex_df in enumerate(split_tex_dfs):
    # only show 3 decimal places
    tex_sty = split_tex_df.style.format(precision=3).hide(axis='index')
    caption = captions[family]
    latex = tex_sty.to_latex(
        column_format='|l|' + (len(tex_sty.columns)-1)*'c|',
        hrules=True,
        caption=caption,
    )
    # make table and caption take up full width
    latex = latex.replace('begin{table}', 'begin{table*}').replace('end{table}', 'end{table*}')

    out_file = out_path / data_path.name.replace('.csv',f'_{family}-{i}.tex')
    f = open(out_file, "w")
    f.write(latex)
    f.close()
    print('wrote file: ', out_file)

wrote file:  out/comprehensive_search_GGA-0.tex
wrote file:  out/comprehensive_search_GGA-1.tex
wrote file:  out/comprehensive_search_MGGA-0.tex
wrote file:  out/comprehensive_search_HYB_GGA-0.tex
wrote file:  out/comprehensive_search_HYB_GGA-1.tex
wrote file:  out/comprehensive_search_HYB_MGGA-0.tex
